<a href="https://colab.research.google.com/github/ansul1214/Open_Project_Winter_2025/blob/main/W1_Quantum_Tomography_23113029.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Task 1: Environment Setup

In [1]:
!python -m pip install  pennylane numpy scipy pandas plotly tqdm nbformat


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.3/934.3 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 66.9 MB/s eta 0:00:00


In [2]:
!python -m pip install jax~=0.6.0 jaxlib~=0.6.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 MB 7.1 MB/s eta 0:00:00
  Attempting uninstall: jaxlib
    Found existing installation: jaxlib 0.7.2
    Uninstalling jaxlib-0.7.2:
      Successfully uninstalled jaxlib-0.7.2
  Attempting uninstall: jax
    Found existing installation: jax 0.7.2
    Uninstalling jax-0.7.2:
      Successfully uninstalled jax-0.7.2


In [3]:
import pennylane as qml
import numpy as np
import json
from typing import Dict, Any

# Task 2: Measurement Theory Pointer

To-Do: Prepare at minimum the computational basis (|0⟩, |1⟩), the Hadamard basis (|+⟩, |−⟩), and one phase-offset state (e.g., ( |0⟩ + i |1⟩ ) / √2). Document how you synthesize each state in circuit form and store a textual or JSON summary of the gates used. You may optionally include mixed states by applying depolarizing or amplitude damping channels.

|0> : [1 0].T - 0 bit Dirac notation


|1> : [0 1].T - 1 bit Dirac notation

Hadmard Basis:

These are the linear combinations of the standard basis |0> and |1>.

|+> = 1/√2(|0>+|1>)

|-> = 1/√2(|0>-|1>)

The |1> can be represented as a circuit of the not operator on the zero bit.

|1>=X|0>

X=array([[1, 0],
       [0, 1]])

For |+> = H|0>

 |-> = HS|0>

 For |0+i1>/√2
   
   |0+i1>/√2 = HS|0>


In [4]:
dev = qml.device("default.qubit", wires=1)

In [5]:
def prepare(state):
    if state == "|0>":
        pass
    elif state == "|1>":
        qml.PauliX(0)
    elif state == "|+>":
        qml.Hadamard(0)
    elif state == "|->":
        qml.PauliX(0)
        qml.Hadamard(0)
    elif state == "|0+i1>/√2":
        qml.Hadamard(0)
        qml.S(0)
    else:
        raise ValueError("Invalid state")

In [6]:
state_preparation_summary = {
    "|0>": [],
    "|1>": ["PauliX"],
    "|+>": ["Hadamard"],
    "|->": ["PauliX", "Hadamard"],
    "|0+i1>/√2": ["Hadamard", "S"]
}

with open("state_preparation.json", "w") as f:
    json.dump(state_preparation_summary, f, indent=2)


In [7]:
@qml.qnode(dev)
def noisy_plus_state(p=0.1):
    qml.Hadamard(0)
    qml.DepolarizingChannel(p, wires=0)
    return qml.density_matrix(wires=0)

In [8]:
import pathlib

def build_measurement_model(config_path: pathlib.Path) -> Dict[str, Any]:
    """
    Stub for constructing or loading the measurement operators you plan to use.
    Populate the return value with operator definitions, normalization checks, and metadata.
    """
    X=np.array([[0,1],[1,0]],dtype=complex)
    Y=np.array([[0,-1j],[1j,0]],dtype=complex)
    Z=np.array([[1,0],[0,-1]],dtype=complex)
    I=np.array([[1,0],[0,1]],dtype=complex)
    P0=np.array([[1,0],[0,0]],dtype=complex)
    P1=np.array([[0,0],[0,1]],dtype=complex)
    completeness=np.closeall(P0+P1,I)
    return {
        "type": "pauli_projective",
        "operators":{"X":X,"Y":Y,"Z":Z,"I":I},
        "Projections":{"P0":P0,"P1":P1},
        "completeness":completeness,
        "metadata": {
            "framework": "PennyLane",
            "description": "Single-qubit Pauli projective measurement model"
        }
    }


    # TODO: implement SIC POVM or Pauli projective operator assembly here.
    raise NotImplementedError("Create your measurement operator assembly here.")

# Task 3: QST Data Generation

# Task 4: Single Qubit Tomography